# Computes the performance of the predictions

In [2]:
from sklearn import metrics as sklearn_metrics
import numpy as np
import PIL.Image as pilimage
import matplotlib.pylab as plt

In [3]:
import os
# open the output file
name_file='REGICOR_67_6Lab_noise.csv'
fitxer=open('/media/HDD3TB/data_carotid/SEGMENTACIO/keras_semantic_segmentation/trained_models_carotida/evaluations/'+name_file,'w')
fitxer.write('accuracy,sensitivity,specificity,precision,jaccard,dice\n')

#directory with the outpu images of the model
path='/media/HDD3TB/data_carotid/SEGMENTACIO/keras_semantic_segmentation/trained_models_carotida/'
model='CAROTIDA_segmentation_Tiramisu_rmsprop_model_T67_noise/test_epoch_335_REGICOR/'

#directory with the GT
gt_path='/media/HDD3TB/data_carotid/SEGMENTACIO/keras_semantic_segmentation/sample_bea/data/test18_2Lab/'

images=np.sort(os.listdir(path+model))
gt_images=np.sort(os.listdir(gt_path))

#initialize the values
out_jac=0.
out_acc=0.
out_sens=0.
out_spec=0.
out_prec=0.
out_dice=0.

for i in range(len(images)):
    im=np.asarray(pilimage.open(path+model+images[i]))
    #select the IMR 
    im=1*(im[:,:,0]==255)* (im[:,:,1]==255)*(im[:,:,2]==0) # 6 labels output
#     im=im[:,:,0]/255                                     # 2 label output
    gt_im=np.asarray(pilimage.open(gt_path+gt_images[i]))
    
    #compute the confusion matrix
    cm = sklearn_metrics.confusion_matrix(gt_im.flatten(), im.flatten())

    tn=cm[0][0]
    fn=cm[1][0]
    tp=cm[1][1]
    fp=cm[0][1]
    
    # compute the metrics using their definitions
    jac=float(tp)/(tp+fp+fn)
    dice=float(2*tp)/(2*tp+fp+fn)
    acc = float(tp+tn) / np.sum(cm)
    sens=float(tp)/(tp+fn)
    spec=float(tn)/(tn+fp)
    prec=float(tp)/(tp+fp)
    
    fitxer.write('{},{},{},{},{},{}\n'.format(acc,sens,spec,prec,jac,dice))
#     print(i,acc,sens,spec,prec,jac,dice)

    #accomulate the values
    out_jac+=jac
    out_acc+=acc
    out_sens+=sens
    out_spec+=spec
    out_prec+=prec
    out_dice+=dice
    
print('mean_values')
print(np.array([out_acc,out_sens,out_spec,out_prec,out_jac,out_dice])/len(images))
fitxer.close()

mean_values
[ 0.98987887  0.88662421  0.99274082  0.78060217  0.70157564  0.82216468]
